In [1]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import model_selection

# Import PySwarms
import pyswarms as ps

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%reload_ext autoreload
%autoreload 2

In [2]:
data = pd.read_csv('parkinsons.csv', delimiter=',')
data.columns

Index(['name', 'MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP',
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA',
       'spread1', 'spread2', 'D2', 'PPE'],
      dtype='object')

In [3]:
# Store the features as X and the labels as y
X = data.drop(columns=['name', 'status']).to_numpy()
y = data['status'].to_numpy()
data.columns

# 2. four best features as per mid sems
X = data[['HNR','RPDE','DFA','PPE']].to_numpy()
X.shape

# # 3. Two-factor components analysis as in file:///home/punny/Downloads/parkinson--published-IJSS%20(1).pdf
# X = data[['MDVP:PPQ', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5', 'MDVP:APQ','Shimmer:DDA']].to_numpy()
# X.shape

# # 4. 2nd best Two-factor components analysis
# X = data[['MDVP:Jitter(%)','MDVP:Jitter(Abs)', 'MDVP:RAP', 'PPE', 'Jitter:DDP','NHR']].to_numpy()
# X.shape

(195, 4)

In [4]:
# 60% training set and 40% testing set
x_train, x_test, y_train, y_test = model_selection.train_test_split (X, y, test_size=0.33, random_state=0)

In [5]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;

In [6]:
print(X.shape)
print(y.shape)

n_inputs_val = X.shape[1]
n_hidden_val = 4
# activation_func= np.tanh(z1)

(195, 4)
(195,)


In [7]:
# Forward propagation
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss. It receives a set of parameters that must be
    rolled-back into the corresponding weights and biases.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """
    # Neural network architecture
    n_inputs = n_inputs_val
    n_hidden = n_hidden_val
    n_classes = 2
    
    a1 = n_inputs_val * n_hidden_val
    a2 = a1 + n_hidden_val
    a3 = a2 + (n_hidden_val * n_classes)
    a4 = a3 + n_classes
    
    # Roll-back the weights and biases
    W1 = params[0:a1].reshape((n_inputs,n_hidden))
    b1 = params[a1:a2].reshape((n_hidden,))
    W2 = params[a2:a3].reshape((n_hidden,n_classes))
    b2 = params[a3:a4].reshape((n_classes,))

    # Perform forward propagation
    z1 = x_train.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = sigmoid(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood
    N = x_train.shape[0] # Number of samples
    corect_logprobs = -np.log(probs[range(N), y_train])
    loss = np.sum(corect_logprobs) / N

    return loss

In [8]:
def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [9]:
%%time
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
dimensions = (n_inputs_val * n_hidden_val) + (n_hidden_val * 2) + n_hidden_val + 2
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=1000)

2019-12-11 01:08:11,948 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0.336
2019-12-11 01:08:27,333 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.3359484248764839, best pos: [  0.08425076   0.07223574   4.57057651  18.23010571  -6.49276763
   1.45028243  -0.57113671   1.29852109   8.84308567   3.33003243
   0.60150122  -2.75931831  46.23828018   1.89772404   1.26871661
   0.86484515  -9.2608946    2.59991491   1.02761573   1.13423863
 -13.24801837   2.17361078   2.65236491   2.2313612    1.66670227
  -5.3127928    1.68283882   2.2277718    3.60169403  -2.4641883 ]


CPU times: user 15.2 s, sys: 346 ms, total: 15.5 s
Wall time: 15.4 s


In [10]:
def predict(X, pos):
    """
    Use the trained weights to perform class predictions.

    Inputs
    ------
    X: numpy.ndarray
        Input Iris dataset
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """
    # Neural network architecture
    n_inputs = n_inputs_val
    n_hidden = n_hidden_val
    n_classes = 2
    
    a1 = n_inputs_val * n_hidden_val
    a2 = a1 + n_hidden_val
    a3 = a2 + (n_hidden_val * n_classes)
    a4 = a3 + n_classes

    # Roll-back the weights and biases
    W1 = pos[0:a1].reshape((n_inputs,n_hidden))
    b1 = pos[a1:a2].reshape((n_hidden,))
    W2 = pos[a2:a3].reshape((n_hidden,n_classes))
    b2 = pos[a3:a4].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = sigmoid(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [11]:
(predict(x_test, pos) == y_test).mean()

0.9384615384615385

In [12]:
n_hidden_val

4